In [ ]:
dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [ ]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp

### Testes Modulares - Referente ao tópico 4 - 4.2 Definição de Grupos Mar Aberto

### Geração de Massa

In [ ]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({'payloadHomol.intermediarias.clienteFoPag':[0],
                             'payloadHomol.intermediarias.principalidadeDeInvestimento':[0],
                             'payloadHomol.intermediarias.flagClienteExistente':[0],
                             'solicitante.saldoMinimo30d':[99]
                            })

In [ ]:
#Criação das massas com casos forçados
massa_definicaoGruposMarAberto = sherlock.Gera_Massa(base_entrada)
massa_definicaoGruposMarAberto.gera_cenario('payloadHomol.intermediarias.clienteFoPag',[0,1])
massa_definicaoGruposMarAberto.gera_cenario('payloadHomol.intermediarias.principalidadeDeInvestimento',[0,1])
massa_definicaoGruposMarAberto.gera_cenario('payloadHomol.intermediarias.flagClienteExistente',[0,1])
massa_definicaoGruposMarAberto.gera_cenario('solicitante.saldoMinimo30d',[-1,0,99,100,101])

massa_definicaoGruposMarAberto.get_massa_final()
massa_definicaoGruposMarAberto.set_bom_arqv()
massa_definicaoGruposMarAberto.converte_massa()

### Execução Modular e Geração de CSV

In [ ]:
listaResultado = rp.execucaoModular(massa_definicaoGruposMarAberto.get_massa_json(),rp.DefinicaoGruposMarAberto)

##### Valida se todas condições estão sendo abordadas

In [ ]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['NovoGrupo'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

In [ ]:
# Renomeação das colunas para o cabeçalho do RMA
massa_definicaoGruposMarAberto.rename_cabecalho()

In [ ]:
massa_definicaoGruposMarAberto.get_massa_dm()

In [ ]:
massa_definicaoGruposMarAberto.atribui_expected(listaResultado,"payloadHomol.intermediarias.NovoGrupo","NovoGrupo")
massa_definicaoGruposMarAberto.get_massa_dm()

### Conversão da massa para CSV

In [ ]:
massa_definicaoGruposMarAberto.get_massa_dm().to_csv('C:/Users/T51657/Desktop/Homologação/Modelos Online/v2/massas/Modulares/massaDefinicaoGruposMarAberto.csv',index=False)